In [1]:
import os
import unittest
import time
from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN
from pymatgen.util.testing import PymatgenTest
from pymatgen.analysis.reaction_network import ReactionNetwork
from pymatgen.analysis.fragmenter import Fragmenter, metal_edge_extender
from pymatgen.entries.mol_entry import MoleculeEntry
from monty.serialization import loadfn
import openbabel as ob
import networkx as nx

In [2]:
from crystal_toolkit import view

In [6]:
reextended_entries = []
entries = loadfn("/Users/samuelblau/pymatgen/test_files/reaction_network_files/LiEC_reextended_entries.json")
for entry in entries:
    if "optimized_molecule" in entry["output"]:
        mol = entry["output"]["optimized_molecule"]
    else:
        mol = entry["output"]["initial_molecule"]
    H = float(entry["output"]["enthalpy"])
    S = float(entry["output"]["entropy"])
    E = float(entry["output"]["final_energy"])
    mol_entry = MoleculeEntry(molecule=mol,energy=E,enthalpy=H,entropy=S,entry_id=entry["task_id"])
    if mol_entry.formula == "Li1":
        if mol_entry.charge == 1:
            reextended_entries.append(mol_entry)
    else:
        reextended_entries.append(mol_entry)

In [7]:
RN = ReactionNetwork(reextended_entries)

682 input entries
677 connected entries
569 unique entries


In [8]:
EC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("/Users/samuelblau/pymatgen/test_files/reaction_network_files/EC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
EC_mg = metal_edge_extender(EC_mg)

LiEC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("/Users/samuelblau/pymatgen/test_files/reaction_network_files/LiEC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LiEC_mg = metal_edge_extender(LiEC_mg)

LEDC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("/Users/samuelblau/pymatgen/test_files/reaction_network_files/LEDC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LEDC_mg = metal_edge_extender(LEDC_mg)

LEMC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("/Users/samuelblau/pymatgen/test_files/reaction_network_files/LEMC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LEMC_mg = metal_edge_extender(LEMC_mg)

EC_ind = None
LiEC_ind = None
LEDC_ind = None
LEMC_ind = None
for entry in RN.entries["C3 H4 O3"][10][0]:
    if EC_mg.isomorphic_to(entry.mol_graph):
        EC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C3 H4 Li1 O3"][12][1]:
    if LiEC_mg.isomorphic_to(entry.mol_graph):
        LiEC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C4 H4 Li2 O6"][17][0]:
    if LEDC_mg.isomorphic_to(entry.mol_graph):
        LEDC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C3 H5 Li1 O4"][13][0]:
    if LEMC_mg.isomorphic_to(entry.mol_graph):
        LEMC_ind = entry.parameters["ind"]
        break
Li1_ind = RN.entries["Li1"][0][1][0].parameters["ind"]

print("EC_ind",EC_ind)
print("LiEC_ind",LiEC_ind)
print("LEDC_ind",LEDC_ind)
print("LEMC_ind",LEMC_ind)
print("Li1_ind",Li1_ind)

EC_ind 456
LiEC_ind 424
LEDC_ind 511
LEMC_ind 469
Li1_ind 556


In [61]:
ind=30
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].formula)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-2.9948617707599996, left=-2.9948617707599996, n…

C1 H2 O3


-7212.1124817911705

In [33]:
ind=164
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-3.2333264612400003, left=-3.2333264612400003, n…

0


-2137.8055071671874

In [62]:
for neighbor in list(RN.graph.neighbors(543)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

543,542
rxn_type : One electron reduction
bipartite : 1
energy : -0.14261502319999997
free_energy : -1.7307530161542464

543,544
rxn_type : One electron oxidation
bipartite : 1
energy : 0.2977278364
free_energy : 6.261056343242997

543+PR_0,3
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.14101248199139949
free_energy : -3.4464913879378027

543+PR_1,4
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.1877284285703985
free_energy : -4.711917748998266

543+PR_2,5
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.14973069604319622
free_energy : -3.6789838993568846

543+PR_33,7
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.10952233905199538
free_energy : -2.585022185395209

543+PR_34,8
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0548811656490108
free_energy : -1.0767564268475862

543+PR_40,10
rxn_type : Molecu

energy : -0.11266353309600052
free_energy : -2.4901910341609064

543+PR_126,158
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0876459919369984
free_energy : -1.8211858396347989

543+PR_125,158
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.15330653424800633
free_energy : -3.605185264313567

543+PR_124,158
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.14531518055000336
free_energy : -3.3559165644387203

543+PR_130,159
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.09503284657299449
free_energy : -2.0338982138914243

543+PR_129,160
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.08538449164800344
free_energy : -1.7263236969997333

543+PR_128,160
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.1622192102250074
free_energy : -3.8044783391209958

543+PR_130,160
rxn_type : 

bipartite : 1
energy : -0.10862801652999454
free_energy : -2.375270554594371

543+PR_268,327
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.06310734111000105
free_energy : -1.197377584787736

543+PR_269,328
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.14780506141901018
free_energy : -3.426658250863019

543+PR_269,329
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.21344641976500955
free_energy : -5.158740088227622

543+PR_270,330
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.22563807175098194
free_energy : -5.478200417745446

543+PR_272,331
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.11580217261999681
free_energy : -2.5333547270297085

543+PR_272,332
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.046801936772997155
free_energy : -0.6679079031623214

543+PR_271,33


543+PR_348,416
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.04033861943199557
free_energy : -0.5394547738801148

543+PR_352,418
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.1572645653500081
free_energy : -3.6455483344602353

543+PR_353,418
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.10130635959896839
free_energy : -2.08126828024111

543+PR_350,418
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.07345116874799718
free_energy : -1.4231636719273038

543+PR_354,419
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.1652034414860032
free_energy : -3.890116098115376

543+PR_355,419
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.14526484457698874
free_energy : -3.3411683339695255

543+PR_356,420
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energ

In [13]:
for neighbor in list(RN.graph.neighbors(474)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

474,473
rxn_type : One electron reduction
bipartite : 1
energy : -0.08791600709999958
free_energy : -0.2423167564408346

474+PR_11,7
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.06301589485099957
free_energy : -1.3908328395788203

474+PR_12,8
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.018394655645999514
free_energy : -0.2105903542338865

474+PR_21,19
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0809593017609922
free_energy : -1.866753251131854

474+PR_23,20
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03034432508300089
free_energy : -0.49996023164806047

474+PR_37,28
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.054634213966999745
free_energy : -1.1508368558471602

474+PR_38,29
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.004152531853997665
free_energy : 

In [15]:
for neighbor in list(RN.graph.neighbors(35)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

35,36
rxn_type : One electron oxidation
bipartite : 1
energy : 0.1816016427779914
free_energy : 2.7270388301945787

35,33+479
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.35794579207203014
free_energy : 9.213829847470151

35,34+478
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.2016542032849884
free_energy : 4.987704475838655

35+PR_4,62
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.09198700893955447
free_energy : -1.9750776559694714

35+PR_4,63
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.09441645627055095
free_energy : -2.026401738522509

35+PR_5,64
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.26247726442579733
free_energy : -6.570088676106934

35+PR_5,65
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.263785856735808
free_energy : -6.63231131435623

35+

In [25]:
ind=35
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-3.2421475638, left=-3.2421475638, near=-2000.0,…

-1


-7389.618314120459

In [27]:
for neighbor in list(RN.graph.neighbors(122)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

122,123
rxn_type : One electron oxidation
bipartite : 1
energy : 0.1816016427779914
free_energy : 2.7270388301945787

122,119
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : 0.13608182594003893
free_energy : 3.5303255496419297

122+PR_4115,3
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.5128249622669756
free_energy : -13.560052950197132

122+PR_4139,4
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.18390994963908724
free_energy : -4.429906424613364

122+PR_4174,29
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0505612299340078
free_energy : -0.7976820247390233

122+PR_4175,31
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2965126610809995
free_energy : -7.776907541486449

122+PR_4175,32
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2892071166939906
free_energy : -7.61447432186034



In [24]:
ind=145
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.10976211932, left=-4.10976211932, near=-2000.…

0


-4166.604920618956

In [48]:
for neighbor in list(RN.graph.neighbors(145)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

145,144
rxn_type : One electron reduction
bipartite : 1
energy : -0.04817687610102439
free_energy : 0.8217637178089716

145+PR_5,265
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.06064451272959559
free_energy : -1.095008192156456

145+PR_48,275
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.046755706438034395
free_energy : -0.6612801349838264

145+PR_90,448
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.07923878441999932
free_energy : -1.5156640899449485

145+PR_151,766
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03331586886201876
free_energy : -0.4402560319958866

145+PR_611,1706
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.07314353301801191
free_energy : -1.437030540590058

145+PR_768,1957
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.032917536861020835
free_

In [26]:
ind=448
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.32244975896, left=-4.32244975896, near=-2000.…

0


-7199.132174674312

In [7]:
print(RN.entries["C1 Li2 O3"])

{6: {1: [MoleculeEntry 116846 - C1 Li2 O3 - E6 - C1
Energy = -278.7979 Hartree
Correction = 0.0000 Hartree
Enthalpy = 15.3870 kcal/mol
Entropy = 79.6660 cal/mol.K
Free Energy = -7586.8394 eV
Parameters:
ind = 42]}, 7: {-1: [MoleculeEntry 116849 - C1 Li2 O3 - E7 - C-1
Energy = -279.0779 Hartree
Correction = 0.0000 Hartree
Enthalpy = 16.1850 kcal/mol
Entropy = 72.5060 cal/mol.K
Free Energy = -7594.3315 eV
Parameters:
ind = 43], 0: [MoleculeEntry 115771 - C1 Li2 O3 - E7 - C0
Energy = -279.0434 Hartree
Correction = 0.0000 Hartree
Enthalpy = 16.3010 kcal/mol
Entropy = 72.8510 cal/mol.K
Free Energy = -7593.3932 eV
Parameters:
ind = 44]}}


In [9]:
ind=44
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.07379685944, left=-4.07379685944, near=-2000.…

0


-7593.393205603747

In [8]:
ind=42
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.802616898319999, left=-4.802616898319999, nea…

1


-7586.839388830761

In [36]:
for neighbor in list(RN.graph.neighbors(557)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

557,556
rxn_type : One electron reduction
bipartite : 1
energy : -0.06941194170000031
free_energy : 0.261204583744012

557,558
rxn_type : One electron oxidation
bipartite : 1
energy : 0.08791600709999958
free_energy : 0.2423167564408346

557+PR_13,6
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03524243855000009
free_energy : -0.6493892486219579

557+PR_14,7
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.07325191736698855
free_energy : -1.7281729546585325

557+PR_16,8
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.11322436809700775
free_energy : -2.8788051533366

557+PR_23,20
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03519803221999496
free_energy : -0.6168890091844901

557+PR_25,21
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.04771402998099461
free_energy : -1.0538827923510894

557+PR_28,22
rxn_t

In [22]:
ind=424
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-5.6984556159599995, left=-5.6984556159599995, n…

1


-9519.538722407473

In [35]:
ind=557
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-5.329070518200751e-16, left=-5.329070518200751e…

0


-203.8840240968338

In [17]:
print(len(RN.identify_sinks()))
len(RN.graph.nodes)

39


11135

In [18]:
RN.identify_sinks()

[11,
 30,
 42,
 109,
 171,
 175,
 188,
 194,
 201,
 204,
 278,
 421,
 433,
 434,
 436,
 439,
 463,
 469,
 470,
 481,
 483,
 508,
 511,
 515,
 517,
 519,
 522,
 525,
 528,
 531,
 532,
 535,
 537,
 541,
 546,
 551,
 554,
 563,
 568]

In [7]:
ind=1665
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].formula

Renderer(background='white', camera=OrthographicCamera(bottom=-5.6984556159599995, left=-5.6984556159599995, n…

1


'C3 H4 Li1 O3'

In [28]:
for ind in RN.identify_sinks():
    print(ind, RN.entries_list[ind].formula,RN.entries_list[ind].charge)

101 C2 H2 Li1 O3 -1
160 C2 H4 Li1 O2 -1
250 C3 H2 Li1 O2 -1
424 C3 H5 Li1 O3 0
427 C3 H5 Li1 O4 0
459 C4 H4 Li2 O6 0
462 C6 H8 Li2 O6 0
470 H2 O1 0


In [9]:
ind=2744
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-11.317179820319998, left=-11.317179820319998, n…

-16910.7035955349

In [12]:
ind=1831
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-8.21378872128, left=-8.21378872128, near=-2000.…

-11587.839161760392

In [51]:
ind=4095
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].free_energy)
print(RN.entries_list[ind].formula, RN.entries_list[ind].charge)

Renderer(background='white', camera=OrthographicCamera(bottom=-1.2663532872, left=-1.2663532872, near=-2000.0,…

-2048.475460259543
H1 O1 1


In [14]:
RN.find_paths_v2([1665],2744,weight="softplus")

Finding initial paths...
0 0


NameError: name 'path_dict' is not defined

In [38]:
for neighbor in list(RN.graph.neighbors(4121)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

4121,4120
rxn_type : One electron reduction
bipartite : 1
energy : -0.25404839339999796
free_energy : -4.763009911680774

9+4121,15
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2726158202719944
free_energy : -8.161269164406121

16+4121,25
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3503256493190321
free_energy : -10.231711858673634

20+4121,26
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2683419403069962
free_energy : -8.01310848356411

22+4121,28
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.26362386189197196
free_energy : -7.9984654849558865

23+4121,30
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3530475554039896
free_energy : -10.2207894938349

26+4121,32
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.1070986751279861
free_energy : -3.8243422996238223

2

rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.27272224830802827
free_energy : -8.181360732686016

729+4121,742
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3186692434870366
free_energy : -9.244234579393606

733+4121,743
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3618918184810269
free_energy : -10.463985348202414

738+4121,745
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.33733379231297533
free_energy : -9.793292773212897

747+4121,752
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2869428221599861
free_energy : -8.46757146477853

156+4121,764
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.34185991947607874
free_energy : -10.036128365784634

157+4121,765
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3963590346541963
free_en

free_energy : -10.275421494396369

1384+4121,1414
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35310175247401787
free_energy : -10.251723672778326

1387+4121,1415
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.22398368141303138
free_energy : -6.861609574287513

1386+4121,1416
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35224229056599654
free_energy : -10.247079350909416

1385+4121,1417
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2692443296609923
free_energy : -8.048937290743424

1272+4121,1473
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2656255738359903
free_energy : -7.898532114501904

1271+4121,1474
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.38348713362400133
free_energy : -11.040456247329075

1272+4121,1476
rxn_type : Molecular formation from one new

bipartite : 1
energy : -0.3028360058699917
free_energy : -8.930733950604463

1891+4121,1893
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3529274418380197
free_energy : -10.241516214568264

1892+4121,1894
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2903307995669735
free_energy : -8.610694955316148

1899+4121,1904
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35376919126896667
free_energy : -10.293057910789003

1901+4121,1906
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.32975495428603097
free_energy : -9.745305288895224

1902+4121,1914
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2928321625199857
free_energy : -8.68543156457622

1902+4121,1915
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3777049614059962
free_energy : -10.888412055417803

1905+4121,1919
rxn_

energy : -0.26176344303598853
free_energy : -7.862735299795531

2280+4121,2344
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.1967339571789637
free_energy : -6.146993570228005

2285+4121,2347
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3532255722919899
free_energy : -10.309120834692294

2282+4121,2348
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3738626368719622
free_energy : -10.786137842216704

2278+4121,2349
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3889311523080039
free_energy : -11.165396636691185

2281+4121,2350
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.22621928369702005
free_energy : -6.849376038890114

2277+4121,2353
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3187894352609817
free_energy : -9.313560830878487

2360+4121,2366
rxn_type : Molecu

rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.41118102117197
free_energy : -11.771673796096593

2784+4121,2877
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.244339169133994
free_energy : -7.457077173900188

2785+4121,2878
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.21895674969098877
free_energy : -6.726023301927853

2782+4121,2882
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.21875528222096818
free_energy : -6.739497101244524

2787+4121,2882
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.4347650237929628
free_energy : -12.46758349946981

2798+4121,2884
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3444866975140002
free_energy : -10.105989079760775

2892+4121,2894
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.352019860991973

rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35313069351299475
free_energy : -10.25149214232647

3474+4121,3491
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35268352462300356
free_energy : -10.26610407559042

3476+4121,3492
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.25170067889497716
free_energy : -7.536882998163719

3485+4121,3493
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35395486704801726
free_energy : -10.255419404384384

3486+4121,3494
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.3528320952289761
free_energy : -10.262913669681666

3504+4121,3517
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35583318518899887
free_energy : -10.33379553591567

3502+4121,3518
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.321763928

free_energy : -10.240930575910397

3970+4121,3991
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35278857147501697
free_energy : -10.2572059318411

3972+4121,3992
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.353101218524003
free_energy : -10.242492884354078

3971+4121,3993
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35313416020495936
free_energy : -10.251571818764432

3973+4121,3993
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35356712891497466
free_energy : -10.254047216418485

3968+4121,3994
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.35528439962594405
free_energy : -10.332827032514842

3974+4121,3994
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.352769817676986
free_energy : -10.2641780036613

3986+4121,3995
rxn_type : Molecular formation from one new bo

In [7]:
view(RN.entries_list[220].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[220].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-5.164007155319999, left=-5.164007155319999, nea…

0

In [8]:
for neighbor in list(RN.graph.neighbors(220)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

220,221
rxn_type : One electron oxidation
bipartite : 1
energy : 0.07843445247800673
free_energy : 2.169687602419799

220,219
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : -0.03746218086303088
free_energy : -1.1988151561727136



In [9]:
view(RN.entries_list[219].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[219].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-6.7520014130399995, left=-6.7520014130399995, n…

0

In [10]:
for neighbor in list(RN.graph.neighbors(219)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

219,218
rxn_type : one_electron_reduction
bipartite : 1
energy : -0.10700326433396867
free_energy : -2.9053784799398272

219,220
rxn_type : intramol_single_bond_formation
bipartite : 1
energy : 0.03746218086303088
free_energy : 1.1988151561727136

219,26+120
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.12262937153201392
free_energy : 2.748628039684263

219,27+119
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.03514357447528482
free_energy : 0.36044270861384575

219,12+85
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.32784811307334394
free_energy : 8.187117081153701

219,11+87
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.18139776248142425
free_energy : 4.205527752190392

219,10+89
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.21144497289822084
free_energy : 5.007263512266945

26+219,245
rx

In [11]:
view(RN.entries_list[218].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[218].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-7.18941198384, left=-7.18941198384, near=-2000.…

-1

In [12]:
for neighbor in list(RN.graph.neighbors(218)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

218,219
rxn_type : one_electron_oxidation
bipartite : 1
energy : 0.10700326433396867
free_energy : 2.9053784799398272

218,10+87
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.15015395331317904
free_energy : 3.330511119633229

218,11+85
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.19501205221041573
free_energy : 4.5662127215728106

218,27+118
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : 0.118908146099443
free_energy : 2.4039168018844066

218,26+119
rxn_type : Molecular decomposition breaking one bond: A->B+C
bipartite : 1
energy : -0.039454803968737906
free_energy : -1.6112176416409056

27+218,245
rxn_type : Molecular formation from one new bond: A+B -> C
bipartite : 1
energy : -0.05106248161604299
free_energy : -0.6037311549762308



In [81]:
ind=70
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[ind].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-2.26998221796, left=-2.26998221796, near=-2000.…

1

In [48]:
ind=87
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[ind].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-5.536899364080001, left=-5.536899364080001, nea…

0

In [15]:
view(RN.entries_list[27].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[27].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-3.04935337968, left=-3.04935337968, near=-2000.…

0

In [23]:
view(RN.entries_list[232].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[232].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-5.890315477560001, left=-5.890315477560001, nea…

-1

In [21]:
for neighbor in list(RN.graph.neighbors(246)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

246,245
rxn_type : One electron reduction
bipartite : 1
energy : -0.01568824934508939
free_energy : -0.45401455634782906

246,27+219
rxn_type : Molecular decomposition breaking one bond A->B+C
bipartite : 1
energy : 0.14237749660492227
free_energy : 3.0550950785691384

246,85+89
rxn_type : Molecular decomposition breaking one bond A->B+C
bipartite : 1
energy : 0.2262870749210606
free_energy : 5.334094910562271

246,87+87
rxn_type : Molecular decomposition breaking one bond A->B+C
bipartite : 1
energy : 0.15138176560697048
free_energy : 3.2966575485479552



In [50]:
to_view = [27, 219, 89, 85, 26, 10, 12, 11, 2, 18, 40, 70]

for ind in to_view:
    view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
    print(ind,RN.entries_list[ind].charge)

Renderer(background='white', camera=OrthographicCamera(bottom=-3.04935337968, left=-3.04935337968, near=-2000.…

27 0


Renderer(background='white', camera=OrthographicCamera(bottom=-6.7520014130399995, left=-6.7520014130399995, n…

219 0


Renderer(background='white', camera=OrthographicCamera(bottom=-5.76610607112, left=-5.76610607112, near=-2000.…

...

18 1


Renderer(background='white', camera=OrthographicCamera(bottom=-2.4544077762, left=-2.4544077762, near=-2000.0,…

40 1


Renderer(background='white', camera=OrthographicCamera(bottom=-2.26998221796, left=-2.26998221796, near=-2000.…

70 1


In [10]:
for ii,entry in enumerate(RN.entries_list):
    print(ii,entry.molecule.formula)
    

0 H1 C1
1 H1 C1
2 H1 C1
3 Li1 H1 C1 O1
4 Li1 H1 C1 O1
5 Li1 H1 C1 O1
6 Li1 H1 C1 O2
7 H1 C1 O1
8 H1 C1 O1
9 H1 C1 O1
10 H2 C1
11 H2 C1
12 H2 C1
13 Li1 H2 C1 O1
14 Li1 H2 C1 O1
15 Li1 H2 C1 O1
16 H2 C1 O1
17 H2 C1 O1
18 H2 C1 O1
19 Li1 C1 O1
20 Li1 C1 O1
21 Li1 C1 O2
22 Li1 C1 O2
23 Li1 C1 O2
24 Li1 C1 O2
25 Li1 C1 O2
26 Li1 C1 O3
27 Li1 C1 O3
28 C1 O1
29 C1 O1
30 C1 O1
31 C1 O2
32 C1 O2
33 C1 O2
34 C1 O3
35 C2
36 C2
37 C2
38 H1 C2
39 H1 C2
40 H1 C2
41 Li1 H1 C2 O1
42 Li1 H1 C2 O1
43 Li1 H1 C2 O1
44 Li1 H1 C2 O1
45 Li1 H1 C2 O2
46 Li1 H1 C2 O2
47 Li1 H1 C2 O2
48 Li1 H1 C2 O2
49 Li1 H1 C2 O2
50 Li1 H1 C2 O2
51 Li1 H1 C2 O2
52 Li1 H1 C2 O2
53 Li1 H1 C2 O3
54 Li1 H1 C2 O3
55 Li1 H1 C2 O3
56 Li1 H1 C2 O3
57 H1 C2 O1
58 H1 C2 O1
59 H1 C2 O1
60 H1 C2 O1
61 H1 C2 O2
62 H1 C2 O2
63 H1 C2 O2
64 H1 C2 O3
65 H2 C2
66 H2 C2
67 H2 C2
68 H2 C2
69 H2 C2
70 H2 C2
71 Li1 H2 C2 O1
72 Li1 H2 C2 O1
73 Li1 H2 C2 O1
74 Li1 H2 C2 O1
75 Li1 H2 C2 O1
76 Li1 H2 C2 O2
77 Li1 H2 C2 O2
78 Li1 H2 C2 O2
79 Li1 H2 C2 

In [14]:
ind=29
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[ind].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-1.2793967673599997, left=-1.2793967673599997, n…

0

In [16]:
ind=245
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
RN.entries_list[ind].charge

Renderer(background='white', camera=OrthographicCamera(bottom=-11.374637609879999, left=-11.374637609879999, n…

-1